# threading - Manage Concurrent Operations Within a Process

Purpose:	Manage several threads of execution.

Using threads allows a program to run multiple operations concurrently in the same process space.

## Thread Objects

The simplest way to use a Thread is to instantiate it with a target function and call start() to let it begin working.

The output is five lines with "Worker" on each.

In [25]:
# threading_simple.py
import threading


def worker():
    """thread worker function"""
    print('Worker')


threads = []
for i in range(5):
    t = threading.Thread(target=worker)
    threads.append(t)
    t.start()

WorkerWorkerWorker
Worker
Worker




In [26]:
threads

[<Thread(Thread-34, stopped 123145564983296)>,
 <Thread(Thread-35, stopped 123145570238464)>,
 <Thread(Thread-36, stopped 123145575493632)>,
 <Thread(Thread-37, stopped 123145564983296)>,
 <Thread(Thread-38, stopped 123145570238464)>]

It is useful to be able to spawn a thread and pass it arguments to tell it what work to do. Any type of object can be passed as argument to the thread. This example passes a number, which the thread then prints.

The integer argument is now included in the message printed by each thread.

In [27]:
# threading_simpleargs.py
import threading


def worker(num):
    """thread worker function"""
    print('Worker: %s' % num)


threads = []
for i in range(5):
    t = threading.Thread(target=worker, args=(i,))
    threads.append(t)
    t.start()

Worker: 0Worker: 1Worker: 2Worker: 3


Worker: 4



## Determining the Current Thread

Using arguments to identify or name the thread is cumbersome and unnecessary. Each Thread instance has a name with a default value that can be changed as the thread is created. Naming threads is useful in server processes with multiple service threads handling different operations.

The debug output includes the name of the current thread on each line. 

In [28]:
# threading_names.py
import threading
import time


def worker():
    print(threading.current_thread().getName(), 'Starting')
    time.sleep(0.2)
    print(threading.current_thread().getName(), 'Exiting')


def my_service():
    print(threading.current_thread().getName(), 'Starting')
    time.sleep(0.3)
    print(threading.current_thread().getName(), 'Exiting')


t = threading.Thread(name='my_service', target=my_service)
w = threading.Thread(name='worker', target=worker)
w2 = threading.Thread(target=worker)  # use default name

w.start()
w2.start()
t.start()

worker Startingmy_serviceThread-44
  StartingStarting



Most programs do not use print to debug. The logging module supports embedding the thread name in every log message using the formatter code %(threadName)s. Including thread names in log messages makes it possible to trace those messages back to their source.

logging is also thread-safe, so messages from different threads are kept distinct in the output.

In [29]:
# threading_names_log.py
import logging
import threading
import time


def worker():
    logging.debug('Starting')
    time.sleep(0.2)
    logging.debug('Exiting')


def my_service():
    logging.debug('Starting')
    time.sleep(0.3)
    logging.debug('Exiting')


logging.basicConfig(
    level=logging.DEBUG,
    format='[%(levelname)s] (%(threadName)-10s) %(message)s',
)

t = threading.Thread(name='my_service', target=my_service)
w = threading.Thread(name='worker', target=worker)
w2 = threading.Thread(target=worker)  # use default name

w.start()
w2.start()
t.start()

[DEBUG] (worker    ) Starting
[DEBUG] (Thread-45 ) Starting
[DEBUG] (my_service) Starting


## Daemon vs. Non-Daemon Threads

Up to this point, the example programs have implicitly waited to exit until all threads have completed their work. Sometimes programs spawn a thread as a daemon that runs without blocking the main program from exiting. Using daemon threads is useful for services where there may not be an easy way to interrupt the thread, or where letting the thread die in the middle of its work does not lose or corrupt data (for example, a thread that generates “heart beats” for a service monitoring tool). To mark a thread as a daemon, pass daemon=True when constructing it or call its set_daemon() method with True. The default is for threads to not be daemons.

The output does not include the "Exiting" message from the daemon thread, since all of the non-daemon threads (including the main thread) exit before the daemon thread wakes up from the sleep() call.

In [30]:
! python3 threading_daemon.py

(daemon    ) Starting
(non-daemon) Starting
(non-daemon) Exiting


[DEBUG] (worker    ) Exiting
[DEBUG] (Thread-45 ) Exiting


Thread-44worker  ExitingExiting



To wait until a daemon thread has completed its work, use the join() method.

Waiting for the daemon thread to exit using join() means it has a chance to produce its "Exiting" message.

In [31]:
! python3 threading_daemon_join.py

my_service Exiting


[DEBUG] (my_service) Exiting


(daemon    ) Starting
(non-daemon) Starting
(non-daemon) Exiting
(daemon    ) Exiting


In [32]:
! python3 threading_daemon_join_timeout.py

(daemon    ) Starting
(non-daemon) Starting
(non-daemon) Exiting
d.isAlive() True


## Enumerating All Threads

It is not necessary to retain an explicit handle to all of the daemon threads in order to ensure they have completed before exiting the main process. enumerate() returns a list of active Thread instances. The list includes the current thread, and since joining the current thread introduces a deadlock situation, it must be skipped.

Because the worker is sleeping for a random amount of time, the output from this program may vary.

In [33]:
! python3 threading_enumerate.py

(Thread-1  ) sleeping 0.40
(Thread-2  ) sleeping 0.30
(Thread-3  ) sleeping 0.50
(MainThread) joining Thread-1
(Thread-2  ) ending
(Thread-1  ) ending
(MainThread) joining Thread-2
(MainThread) joining Thread-3
(Thread-3  ) ending


## Subclassing Thread

At start-up, a Thread does some basic initialization and then calls its run() method, which calls the target function passed to the constructor. To create a subclass of Thread, override run() to do whatever is necessary.

In [34]:
# threading_subclass.py
import threading
import logging


class MyThread(threading.Thread):

    def run(self):
        logging.debug('running')


logging.basicConfig(
    level=logging.DEBUG,
    format='(%(threadName)-10s) %(message)s',
)

for i in range(5):
    t = MyThread()
    t.start()

[DEBUG] (Thread-46 ) running
[DEBUG] (Thread-47 ) running
[DEBUG] (Thread-48 ) running
[DEBUG] (Thread-49 ) running
[DEBUG] (Thread-50 ) running


Because the args and kwargs values passed to the Thread constructor are saved in private variables using names prefixed with '\__', they are not easily accessed from a subclass. To pass arguments to a custom thread type, redefine the constructor to save the values in an instance attribute that can be seen in the subclass.

MyThreadWithArgs uses the same API as Thread, but another class could easily change the constructor method to take more or different arguments more directly related to the purpose of the thread, as with any other class.

In [35]:
# threading_subclass_args.py
import threading
import logging


class MyThreadWithArgs(threading.Thread):

    def __init__(self, group=None, target=None, name=None,
                 args=(), kwargs=None, *, daemon=None):
        super().__init__(group=group, target=target, name=name,
                         daemon=daemon)
        self.args = args
        self.kwargs = kwargs

    def run(self):
        logging.debug('running with %s and %s',
                      self.args, self.kwargs)


logging.basicConfig(
    level=logging.DEBUG,
    format='(%(threadName)-10s) %(message)s',
)

for i in range(5):
    t = MyThreadWithArgs(args=(i,), kwargs={i+10: 'A', i+20: 'B'})
    t.start()


[DEBUG] (Thread-51 ) running with (0,) and {10: 'A', 20: 'B'}
[DEBUG] (Thread-52 ) running with (1,) and {11: 'A', 21: 'B'}
[DEBUG] (Thread-53 ) running with (2,) and {12: 'A', 22: 'B'}

## Timer Threads

One example of a reason to subclass Thread is provided by Timer, also included in threading. A Timer starts its work after a delay, and can be canceled at any point within that delay time period.

The second timer in this example is never run, and the first timer appears to run after the rest of the main program is done. Since it is not a daemon thread, it is joined implicitly when the main thread is done.

In [36]:
# threading_timer.py
import threading
import time
import logging


def delayed():
    logging.debug('worker running')


logging.basicConfig(
    level=logging.DEBUG,
    format='(%(threadName)-10s) %(message)s',
)

t1 = threading.Timer(0.3, delayed)
t1.setName('t1')
t2 = threading.Timer(0.3, delayed)
t2.setName('t2')

logging.debug('starting timers')
t1.start()
t2.start()

logging.debug('waiting before canceling %s', t2.getName())
time.sleep(0.2)
logging.debug('canceling %s', t2.getName())
t2.cancel()
logging.debug('done')


[DEBUG] (Thread-54 ) running with (3,) and {13: 'A', 23: 'B'}
[DEBUG] (Thread-55 ) running with (4,) and {14: 'A', 24: 'B'}
[DEBUG] (MainThread) starting timers
[DEBUG] (MainThread) waiting before canceling t2
[DEBUG] (MainThread) canceling t2
[DEBUG] (MainThread) done


## Signaling Between Threads

Although the point of using multiple threads is to run separate operations concurrently, there are times when it is important to be able to synchronize the operations in two or more threads. Event objects are a simple way to communicate between threads safely. An Event manages an internal flag that callers can control with the set() and clear() methods. Other threads can use wait() to pause until the flag is set, effectively blocking progress until allowed to continue.

The wait() method takes an argument representing the number of seconds to wait for the event before timing out. It returns a Boolean indicating whether or not the event is set, so the caller knows why wait() returned. The is_set() method can be used separately on the event without fear of blocking.

In this example, wait_for_event_timeout() checks the event status without blocking indefinitely. The wait_for_event() blocks on the call to wait(), which does not return until the event status changes.

In [37]:
# threading_event.py
import logging
import threading
import time


def wait_for_event(e):
    """Wait for the event to be set before doing anything"""
    logging.debug('wait_for_event starting')
    event_is_set = e.wait()
    logging.debug('event set: %s', event_is_set)


def wait_for_event_timeout(e, t):
    """Wait t seconds and then timeout"""
    while not e.is_set():
        logging.debug('wait_for_event_timeout starting')
        event_is_set = e.wait(t)
        logging.debug('event set: %s', event_is_set)
        if event_is_set:
            logging.debug('processing event')
        else:
            logging.debug('doing other work')


logging.basicConfig(
    level=logging.DEBUG,
    format='(%(threadName)-10s) %(message)s',
)

e = threading.Event()
t1 = threading.Thread(
    name='block',
    target=wait_for_event,
    args=(e,),
)
t1.start()

t2 = threading.Thread(
    name='nonblock-A',
    target=wait_for_event_timeout,
    args=(e, 0.2),
)
t2.start()

t3 = threading.Thread(
    name='nonblock-B',
    target=wait_for_event_timeout,
    args=(e, 2),
)
t3.start()

logging.debug('Waiting before calling Event.set()')
time.sleep(0.3)
e.set()
logging.debug('Event is set')

[DEBUG] (block     ) wait_for_event starting
[DEBUG] (nonblock-A) wait_for_event_timeout starting
[DEBUG] (nonblock-B) wait_for_event_timeout starting
[DEBUG] (MainThread) Waiting before calling Event.set()
[DEBUG] (t1        ) worker running
[DEBUG] (nonblock-A) event set: False
[DEBUG] (nonblock-A) doing other work
[DEBUG] (nonblock-A) wait_for_event_timeout starting
[DEBUG] (MainThread) Event is set
[DEBUG] (block     ) event set: True
[DEBUG] (nonblock-B) event set: True


## Controlling Access to Resources

In addition to synchronizing the operations of threads, it is also important to be able to control access to shared resources to prevent corruption or missed data. Python’s built-in data structures (lists, dictionaries, etc.) are thread-safe as a side-effect of having atomic byte-codes for manipulating them (the global interpreter lock used to protect Python’s internal data structures is not released in the middle of an update). Other data structures implemented in Python, or simpler types like integers and floats, do not have that protection. To guard against simultaneous access to an object, use a Lock object.

In this example, the worker() function increments a Counter instance, which manages a Lock to prevent two threads from changing its internal state at the same time. If the Lock was not used, there is a possibility of missing a change to the value attribute.

In [38]:
! python3 threading_lock.py

[DEBUG] (nonblock-A) event set: True
[DEBUG] (nonblock-B) processing event
[DEBUG] (nonblock-A) processing event


(Thread-1  ) Sleeping 0.65
(Thread-2  ) Sleeping 0.42
(MainThread) Waiting for worker threads
(Thread-2  ) Waiting for lock
(Thread-2  ) Acquired lock
(Thread-2  ) Lock Released
(Thread-2  ) Sleeping 0.76
(Thread-1  ) Waiting for lock
(Thread-1  ) Acquired lock
(Thread-1  ) Lock Released
(Thread-1  ) Sleeping 0.35
(Thread-1  ) Waiting for lock
(Thread-1  ) Acquired lock
(Thread-1  ) Lock Released
(Thread-1  ) Done
(Thread-2  ) Waiting for lock
(Thread-2  ) Acquired lock
(Thread-2  ) Lock Released
(Thread-2  ) Done
(MainThread) Counter: 4


To find out whether another thread has acquired the lock without holding up the current thread, pass False for the blocking argument to acquire(). In the next example, worker() tries to acquire the lock three separate times and counts how many attempts it has to make to do so. In the mean time, lock_holder() cycles between holding and releasing the lock, with short pauses in each state used to simulate load.

It takes worker() more than three iterations to acquire the lock three separate times.

In [39]:
! python3 threading_lock_noblock.py

(LockHolder) Starting
(LockHolder) Holding
(Worker    ) Starting
(Worker    ) Trying to acquire
(Worker    ) Iteration 1: Not acquired
(LockHolder) Not holding
(Worker    ) Trying to acquire
(Worker    ) Iteration 2: Acquired
(LockHolder) Holding
(Worker    ) Trying to acquire
(Worker    ) Iteration 3: Not acquired
(LockHolder) Not holding
(Worker    ) Trying to acquire
(LockHolder) Holding
(Worker    ) Iteration 4: Not acquired
(LockHolder) Not holding
(Worker    ) Trying to acquire
(Worker    ) Iteration 5: Acquired
(LockHolder) Holding
(Worker    ) Trying to acquire
(Worker    ) Iteration 6: Not acquired
(LockHolder) Not holding
(Worker    ) Trying to acquire
(Worker    ) Iteration 7: Acquired
(Worker    ) Done after 7 iterations


## Re-entrant Locks

Normal Lock objects cannot be acquired more than once, even by the same thread. This can introduce undesirable side-effects if a lock is accessed by more than one function in the same call chain.

In this case, the second call to acquire() is given a zero timeout to prevent it from blocking because the lock has been obtained by the first call.

In [40]:
# threading_lock_reacquire.py
import threading

lock = threading.Lock()

print('First try :', lock.acquire())
print('Second try:', lock.acquire(0))

First try : True
Second try: False


In a situation where separate code from the same thread needs to “re-acquire” the lock, use an RLock instead.

The only change to the code from the previous example was substituting RLock for Lock.

In [41]:
# threading_rlock.py
import threading

lock = threading.RLock()

print('First try :', lock.acquire())
print('Second try:', lock.acquire(0))

First try : True
Second try: True


## Locks as Context Managers

Locks implement the context manager API and are compatible with the with statement. Using with removes the need to explicitly acquire and release the lock.

The two functions worker_with() and worker_no_with() manage the lock in equivalent ways.

In [42]:
# threading_lock_with.py
import threading
import logging


def worker_with(lock):
    with lock:
        logging.debug('Lock acquired via with')


def worker_no_with(lock):
    lock.acquire()
    try:
        logging.debug('Lock acquired directly')
    finally:
        lock.release()


logging.basicConfig(
    level=logging.DEBUG,
    format='(%(threadName)-10s) %(message)s',
)

lock = threading.Lock()
w = threading.Thread(target=worker_with, args=(lock,))
nw = threading.Thread(target=worker_no_with, args=(lock,))

w.start()
nw.start()

[DEBUG] (Thread-58 ) Lock acquired via with
[DEBUG] (Thread-59 ) Lock acquired directly


## Synchronizing Threads

In addition to using Events, another way of synchronizing threads is through using a Condition object. Because the Condition uses a Lock, it can be tied to a shared resource, allowing multiple threads to wait for the resource to be updated. In this example, the consumer() threads wait for the Condition to be set before continuing. The producer() thread is responsible for setting the condition and notifying the other threads that they can continue.

The threads use with to acquire the lock associated with the Condition. Using the acquire() and release() methods explicitly also works.

In [43]:
! python3 threading_condition.py

2017-07-03 07:46:46,035 (c1) Starting consumer thread
2017-07-03 07:46:46,240 (c2) Starting consumer thread
2017-07-03 07:46:46,444 (p ) Starting producer thread
2017-07-03 07:46:46,445 (p ) Making resource available
2017-07-03 07:46:46,445 (c1) Resource is available to consumer
2017-07-03 07:46:46,445 (c2) Resource is available to consumer


Barriers are another thread synchronization mechanism. A Barrier establishes a control point and all participating threads block until all of the participating “parties” have reached that point. It lets threads start up separately and then pause until they are all ready to proceed.

In this example, the Barrier is configured to block until three threads are waiting. When the condition is met, all of the threads are released past the control point at the same time. The return value from wait() indicates the number of the party being released, and can be used to limit some threads from taking an action like cleaning up a shared resource.

In [44]:
! python3 threading_barrier.py

worker-0 starting
worker-0 waiting for barrier with 0 others
worker-1 starting
worker-1 waiting for barrier with 1 others
worker-2 starting
worker-2 waiting for barrier with 2 others
worker-2 after barrier 2
worker-1 after barrier 1
worker-0 after barrier 0


The abort() method of Barrier causes all of the waiting threads to receive a BrokenBarrierError. This allows threads to clean up if processing is stopped while they are blocked on wait().

This example configures the Barrier to expect one more participating thread than is actually started so that processing in all of the threads is blocked. The abort() call raises an exception in each blocked thread.

In [45]:
! python3 threading_barrier_abort.py

worker-0 starting
worker-0 waiting for barrier with 0 others
worker-1 starting
worker-1 waiting for barrier with 1 others
worker-2 starting
worker-2 waiting for barrier with 2 others
worker-0 aborting
worker-1 aborting
worker-2 aborting


## Limiting Concurrent Access to Resources

Sometimes it is useful to allow more than one worker access to a resource at a time, while still limiting the overall number. For example, a connection pool might support a fixed number of simultaneous connections, or a network application might support a fixed number of concurrent downloads. A Semaphore is one way to manage those connections.

In [46]:
# threading_semaphore.py
import logging
import random
import threading
import time


class ActivePool:

    def __init__(self):
        super(ActivePool, self).__init__()
        self.active = []
        self.lock = threading.Lock()

    def makeActive(self, name):
        with self.lock:
            self.active.append(name)
            logging.debug('Running: %s', self.active)

    def makeInactive(self, name):
        with self.lock:
            self.active.remove(name)
            logging.debug('Running: %s', self.active)


def worker(s, pool):
    logging.debug('Waiting to join the pool')
    with s:
        name = threading.current_thread().getName()
        pool.makeActive(name)
        time.sleep(0.1)
        pool.makeInactive(name)


logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s (%(threadName)-2s) %(message)s',
)

pool = ActivePool()
s = threading.Semaphore(2)
for i in range(4):
    t = threading.Thread(
        target=worker,
        name=str(i),
        args=(s, pool),
    )
    t.start()

[DEBUG] (0         ) Waiting to join the pool
[DEBUG] (1         ) Waiting to join the pool
[DEBUG] (2         ) Waiting to join the pool
[DEBUG] (3         ) Waiting to join the pool


In this example, the ActivePool class simply serves as a convenient way to track which threads are able to run at a given moment. A real resource pool would allocate a connection or some other value to the newly active thread, and reclaim the value when the thread is done. Here, it is just used to hold the names of the active threads to show that at most two are running concurrently.



## Thread-specific Data

While some resources need to be locked so multiple threads can use them, others need to be protected so that they are hidden from threads that do not own them. The local() class creates an object capable of hiding values from view in separate threads.

The attribute local_data.value is not present for any thread until it is set in that thread.

In [47]:
# threading_local.py
import random
import threading
import logging


def show_value(data):
    try:
        val = data.value
    except AttributeError:
        logging.debug('No value yet')
    else:
        logging.debug('value=%s', val)


def worker(data):
    show_value(data)
    data.value = random.randint(1, 100)
    show_value(data)


logging.basicConfig(
    level=logging.DEBUG,
    format='(%(threadName)-10s) %(message)s',
)

local_data = threading.local()
show_value(local_data)
local_data.value = 1000
show_value(local_data)

for i in range(2):
    t = threading.Thread(target=worker, args=(local_data,))
    t.start()

[DEBUG] (0         ) Running: ['0']
[DEBUG] (1         ) Running: ['0', '1']
[DEBUG] (MainThread) No value yet
[DEBUG] (MainThread) value=1000
[DEBUG] (Thread-60 ) No value yet
[DEBUG] (Thread-61 ) No value yet
[DEBUG] (Thread-60 ) value=12
[DEBUG] (Thread-61 ) value=36


To initialize the settings so all threads start with the same value, use a subclass and set the attributes in \__init\__().

\__init\__() is invoked on the same object (note the id() value), once in each thread to set the default values.

In [48]:
# threading_local_defaults.py
import random
import threading
import logging


def show_value(data):
    try:
        val = data.value
    except AttributeError:
        logging.debug('No value yet')
    else:
        logging.debug('value=%s', val)


def worker(data):
    show_value(data)
    data.value = random.randint(1, 100)
    show_value(data)


class MyLocal(threading.local):

    def __init__(self, value):
        super().__init__()
        logging.debug('Initializing %r', self)
        self.value = value


logging.basicConfig(
    level=logging.DEBUG,
    format='(%(threadName)-10s) %(message)s',
)

local_data = MyLocal(1000)
show_value(local_data)

for i in range(2):
    t = threading.Thread(target=worker, args=(local_data,))
    t.start()


[DEBUG] (0         ) Running: ['1']
[DEBUG] (MainThread) Initializing <__main__.MyLocal object at 0x10bf49a08>
[DEBUG] (2         ) Running: ['1', '2']
[DEBUG] (MainThread) value=1000
[DEBUG] (1         ) Running: ['2']
[DEBUG] (Thread-62 ) Initializing <__main__.MyLocal object at 0x10bf49a08>
[DEBUG] (Thread-63 ) Initializing <__main__.MyLocal object at 0x10bf49a08>
[DEBUG] (3         ) Running: ['2', '3']
[DEBUG] (Thread-62 ) value=1000
[DEBUG] (Thread-63 ) value=1000
[DEBUG] (Thread-62 ) value=15
[DEBUG] (Thread-63 ) value=65
[DEBUG] (2         ) Running: ['3']
[DEBUG] (3         ) Running: []
